
Este código utiliza a API do Kaggle para baixar o dataset **"Olist Brazilian E-Commerce"** diretamente para o diretório especificado no computador. O dataset é extraído e salvo no caminho raw.


In [1]:
# Carregas as bibliotecas necessárias
import kaggle
import os
import pandas as pd
import sqlite3

# Nome do dataset extarido do Kaggle
dataset_name = "olistbr/brazilian-ecommerce"

# Define o diretório onde salvar
save_path = r"C:\GitHub\Projeto-Power-BI-Data-Analysis\data\raw"

# Baixa o dataset, usando a api
kaggle.api.dataset_download_files(dataset_name, path=save_path, unzip=True)

print("Download concluído! Arquivos salvos em:", save_path)


Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
Download concluído! Arquivos salvos em: C:\GitHub\Projeto-Power-BI-Data-Analysis\data\raw


#### Lista os arquivos baixados

In [2]:
# Lista os arquivos no diretório
files = os.listdir(save_path)

# Exibe os arquivos
print("Arquivos no diretório:", save_path)
for file in files:
    print(file)

Arquivos no diretório: C:\GitHub\Projeto-Power-BI-Data-Analysis\data\raw
olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_orders_dataset.csv
olist_order_items_dataset.csv
olist_order_payments_dataset.csv
olist_order_reviews_dataset.csv
olist_products_dataset.csv
olist_sellers_dataset.csv
product_category_name_translation.csv


### Renomeando os arquivos, assim deixamos o padrão Staging Area

In [3]:
# Diretório onde os arquivos estão localizados
directory = r"C:\GitHub\Projeto-Power-BI-Data-Analysis\data\raw"

# Listar todos os arquivos no diretório
for filename in os.listdir(directory):
    # Verificar se é um arquivo CSV
    if filename.endswith(".csv"):
        # Criar o novo nome com o prefixo "stg_"
        new_name = "stg_" + filename
        
        # Obter o caminho completo dos arquivos
        old_file = os.path.join(directory, filename)
        new_file = os.path.join(directory, new_name)
        
        # Renomear o arquivo
        os.rename(old_file, new_file)
        print(f"Renomeado: {filename} ---> {new_name}")


Renomeado: olist_customers_dataset.csv ---> stg_olist_customers_dataset.csv
Renomeado: olist_geolocation_dataset.csv ---> stg_olist_geolocation_dataset.csv
Renomeado: olist_orders_dataset.csv ---> stg_olist_orders_dataset.csv
Renomeado: olist_order_items_dataset.csv ---> stg_olist_order_items_dataset.csv
Renomeado: olist_order_payments_dataset.csv ---> stg_olist_order_payments_dataset.csv
Renomeado: olist_order_reviews_dataset.csv ---> stg_olist_order_reviews_dataset.csv
Renomeado: olist_products_dataset.csv ---> stg_olist_products_dataset.csv
Renomeado: olist_sellers_dataset.csv ---> stg_olist_sellers_dataset.csv
Renomeado: product_category_name_translation.csv ---> stg_product_category_name_translation.csv


Carrega os dados da Staging Area para um banco de dados do sqlite3

In [4]:
# Caminho para o banco de dados SQLite
db_path = r"C:\GitHub\Projeto-Power-BI-Data-Analysis\data\processed\stg_data.db"

# Conecta ao banco de dados (ou cria um novo se não existir)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Diretório com os arquivos CSV
directory = r"C:\GitHub\Projeto-Power-BI-Data-Analysis\data\raw"

# Lista os arquivos CSV no diretório
csv_files = [f for f in os.listdir(directory) if f.endswith(".csv")]

# Função para carregar arquivos CSV em uma tabela SQLite
def load_csv_to_sqlite(file_path, table_name):
    try:
        # Carrega o arquivo CSV em um DataFrame
        df = pd.read_csv(file_path)
        
        # Verifica se o DataFrame não está vazio antes de carregar
        if df.empty:
            print(f"Atenção: O arquivo {file_path} está vazio e não será carregado.")
        else:
            # Carrega os dados no banco de dados (tabela de staging)
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            print(f"Tabela {table_name} carregada com sucesso!")
    except Exception as e:
        print(f"Erro ao carregar {file_path} para a tabela {table_name}: {e}")

# Carrega todos os arquivos CSV em tabelas com prefixo 'stg_'
for file in csv_files:
    file_path = os.path.join(directory, file)
    table_name = "stg_" + file.replace(".csv", "")  # Adiciona o prefixo 'stg_'
    load_csv_to_sqlite(file_path, table_name)

# Fechar a conexão
conn.close()


Tabela stg_stg_olist_customers_dataset carregada com sucesso!
Tabela stg_stg_olist_geolocation_dataset carregada com sucesso!
Tabela stg_stg_olist_orders_dataset carregada com sucesso!
Tabela stg_stg_olist_order_items_dataset carregada com sucesso!
Tabela stg_stg_olist_order_payments_dataset carregada com sucesso!
Tabela stg_stg_olist_order_reviews_dataset carregada com sucesso!
Tabela stg_stg_olist_products_dataset carregada com sucesso!
Tabela stg_stg_olist_sellers_dataset carregada com sucesso!
Tabela stg_stg_product_category_name_translation carregada com sucesso!


***Obtém todas as tabelas do banco de dados stg*** \
***Para cada tabela, exibe as colunas e as primeiras 5 linhas*** \
***Garante que todas as tabelas foram carregadas corretamente***


In [ ]:
import sqlite3
import pandas as pd

# Caminho do banco de dados
db_path = r"C:\GitHub\Projeto-Power-BI-Data-Analysis\data\processed\stg_data.db"

# Reabrindo a conexão para consultar o banco
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Obtendo a lista de tabelas no banco de dados
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]

# Percorrendo todas as tabelas e exibindo algumas linhas de cada uma
for table in tables:
    print(f"\n Tabela: {table} \n")

    # Lendo os dados da tabela usando pandas
    df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 5", conn)

    # Exibindo o dataframe como tabela formatada
    print(df.to_markdown(index=False))  # Usa o tabulate (to_markdown) para formatar a saída dos dados

# Fechando a conexão
conn.close()



 Tabela: stg_stg_olist_customers_dataset 

| customer_id                      | customer_unique_id               |   customer_zip_code_prefix | customer_city         | customer_state   |
|:---------------------------------|:---------------------------------|---------------------------:|:----------------------|:-----------------|
| 06b8999e2fba1a1fbc88172c00ba8bc7 | 861eff4711a542e4b93843c6dd7febb0 |                      14409 | franca                | SP               |
| 18955e83d337fd6b2def6b18a428ac77 | 290c77bc529b7ac935b93aa66c333dc3 |                       9790 | sao bernardo do campo | SP               |
| 4e7b3e00288586ebd08712fdd0374a03 | 060e732b5b29e8181a18229c7b0b2b5e |                       1151 | sao paulo             | SP               |
| b2b6027bc5c5109e529d4dc6358b12c3 | 259dac757896d24d7702b9acbbff3f3c |                       8775 | mogi das cruzes       | SP               |
| 4f2d8ab171c80ec8364f7c12e35b23ad | 345ecd01c38d18a9036ed96c73b8d066 |                     